# Project Metadata & Setup

---

## **Project Title:** **Early Breast Cancer Diagnosis using Machine Learning (Terminal-Based Prototype)**

---

### **Objective:**

Develop a lightweight, terminal-driven diagnostic prototype that predicts breast tumor malignancy using the built-in Breast Cancer Wisconsin dataset.
The goal is to simulate how a clinician or technician might use a fast, interpretable tool, without a full GUI or web app.


### **Dataset Description:**

* **Name:** Breast Cancer Wisconsin (Diagnostic)
* **Source:** `sklearn.datasets.load_breast_cancer()`
* **Samples:** 569
* **Features:** 30 numerical (e.g. radius, texture, symmetry)
* **Target:** Binary classification — `malignant (0)` vs. `benign (1)`



### **Stakeholders:**

| Stakeholder          | Interest / Use Case                                              |
| -------------------- | ---------------------------------------------------------------- |
| **Clinicians**       | Fast, interpretable predictions without cloud dependency         |
| **Researchers**      | Baseline model to compare with deep learning or ensemble methods |
| **Medical Startups** | Prototype backend logic for terminal-based tools                 |
| **Educators**        | Teaching ML with meaningful, real-world data                     |


### **Key Notes for Revision:**

* **No GPU required** — uses efficient `RandomForestClassifier`
* **Runs locally** in terminal with zero external dataset download
* **Emphasis on simplicity**: ASCII feedback, clean input prompts
* **Can be modularized later** into API or UI backend
* **We’ll skip test/train splitting** for now — prototype phase only


### **Environment Requirements:**

| Package    | Use                          |
| ---------- | ---------------------------- |
| `sklearn`  | Dataset + ML model           |
| `pandas`   | Tabular data inspection      |
| `colorama` | Terminal coloring (optional) |
| `numpy`    | Data handling                |

---

### Install (if not present):

```bash
pip install pandas scikit-learn colorama
```


### Deliverables:

* Classifier trained on breast cancer dataset
* Real-time terminal prediction system
* ASCII output or basic feedback UI for predicted diagnosis
* Clean, commented code blocks
* Professional markdown formatting with revision comments
